# IF3170 Artificial Intelligence | Praktikum

This notebook serves as a template for the assignment. Please create a copy of this notebook to complete your work. You can add more code blocks, markdown blocks, or new sections if needed.


Group Number: 21

Group Members:
- Denise Felicia Tiowanni (13522013)
- Erdianti WIga Putri Andini (13522053)

## Import Libraries

In [2]:
import pandas as pd
import numpy as np

# Import other libraries if needed

## Import Dataset

In [3]:
# import data dari google drive
test_data = 'https://drive.google.com/uc?id=1k7kmkB-3vkrU5CTDhEVBOxV2VXrnL5ue'
train_data = 'https://drive.google.com/uc?id=1BE2Wt0hd0QZTcHWDJN1f94in-Kq0C6AO'

test = pd.read_csv(test_data)
train = pd.read_csv(train_data)

# 1. Exploratory Data Analysis

Exploratory Data Analysis (EDA) is a crucial step in the data analysis process that involves examining and visualizing data sets to uncover patterns, trends, anomalies, and insights. It is the first step before applying more advanced statistical and machine learning techniques. EDA helps you to gain a deep understanding of the data you are working with, allowing you to make informed decisions and formulate hypotheses for further analysis.

In [ ]:
# Write your code here

# 2. Split Training Set and Validation Set

Splitting the training and validation set works as an early diagnostic towards the performance of the model we train. This is done before the preprocessing steps to **avoid data leakage inbetween the sets**. If you want to use k-fold cross-validation, split the data later and do the cleaning and preprocessing separately for each split.

Note: For training, you should use the data contained in the `train.csv` given by the TA. The `test.csv` data is only used for kaggle submission.

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [14]:
# import data dari google drive
test_data = 'https://drive.google.com/uc?id=1k7kmkB-3vkrU5CTDhEVBOxV2VXrnL5ue'
train_data = 'https://drive.google.com/uc?id=1BE2Wt0hd0QZTcHWDJN1f94in-Kq0C6AO'

test = pd.read_csv(test_data)
train = pd.read_csv(train_data)

In [17]:
target_col = ['Status']
cat_columns = train.select_dtypes(include=['object']).columns.tolist()
indices_to_drop = [i for i, val in enumerate(cat_columns) if val in target_col]
cat_columns = np.delete(cat_columns, indices_to_drop)
num_columns = train.select_dtypes(include=['int64', 'float64']).columns.tolist()   

# split data
x_train = train.drop(target_col, axis=1)
y_train = train[target_col]

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

print(f"Training Set Size: {x_train.shape}, Validation Set Size: {x_val.shape}")


Training Set Size: (12000, 19), Validation Set Size: (3000, 19)


# 3. Data Cleaning and Preprocessing

This step is the first thing to be done once a Data Scientist have grasped a general knowledge of the data. Raw data is **seldom ready for training**, therefore steps need to be taken to clean and format the data for the Machine Learning model to interpret.

By performing data cleaning and preprocessing, you ensure that your dataset is ready for model training, leading to more accurate and reliable machine learning results. These steps are essential for transforming raw data into a format that machine learning algorithms can effectively learn from and make predictions.

For each step that you will do, **please explain the reason why did you do that process. Write it in a markdown cell under the code cell you wrote.**

In [8]:
print("train data:")
print(train.info())

print("\ntest data:")
print(test.info())

train data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             15000 non-null  int64  
 1   N_Days         15000 non-null  float64
 2   Drug           8450 non-null   object 
 3   Age            15000 non-null  float64
 4   Sex            15000 non-null  object 
 5   Ascites        8453 non-null   object 
 6   Hepatomegaly   8448 non-null   object 
 7   Spiders        8441 non-null   object 
 8   Edema          15000 non-null  object 
 9   Bilirubin      15000 non-null  float64
 10  Cholesterol    6626 non-null   float64
 11  Albumin        15000 non-null  float64
 12  Copper         8340 non-null   float64
 13  Alk_Phos       8444 non-null   float64
 14  SGOT           8441 non-null   float64
 15  Tryglicerides  6575 non-null   float64
 16  Platelets      14416 non-null  float64
 17  Prothrombin    14984 non-null  float64

In [7]:
# check for missing values
missing_train = train.isnull().sum()
missing_test = test.isnull().sum()

print("missing values in train data:")
print(missing_train[missing_train > 0])

# percentage
for column in missing_train[missing_train > 0].index:
    print(f"{column}: {missing_train[column] / train.shape[0] * 100:.2f}%")

print("\nmissing values in test data:")
print(missing_test[missing_test > 0])

missing values in train data:
Drug             6550
Ascites          6547
Hepatomegaly     6552
Spiders          6559
Cholesterol      8374
Copper           6660
Alk_Phos         6556
SGOT             6559
Tryglicerides    8425
Platelets         584
Prothrombin        16
dtype: int64

missing values in test data:
N_Days              1
Drug             4368
Ascites          4365
Hepatomegaly     4372
Spiders          4373
Cholesterol      5625
Copper           4440
Alk_Phos         4374
SGOT             4374
Tryglicerides    5657
Platelets         390
Prothrombin        10
dtype: int64


In [9]:
# separate categorical and numerical features
categorical_features = train.select_dtypes(include=['object']).columns.tolist()
numerical_features = train.select_dtypes(include=['int64', 'float64']).columns.tolist()

print("categorical features:")
print(categorical_features)

print("\nnumerical features:")
print(numerical_features)


categorical features:
['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema', 'Status']

numerical features:
['id', 'N_Days', 'Age', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin', 'Stage']


In [10]:
# check unique values in categorical features
for col in categorical_features:
    print(f"unique values in {col}:")
    print(train[col].value_counts())
    print()

unique values in Drug:
Drug
D-penicillamine    4515
Placebo            3935
Name: count, dtype: int64

unique values in Sex:
Sex
F    14405
M      595
Name: count, dtype: int64

unique values in Ascites:
Ascites
N    8027
Y     426
Name: count, dtype: int64

unique values in Hepatomegaly:
Hepatomegaly
N        4652
Y        3795
158.0       1
Name: count, dtype: int64

unique values in Spiders:
Spiders
N    6598
Y    1843
Name: count, dtype: int64

unique values in Edema:
Edema
N    13874
S      737
Y      389
Name: count, dtype: int64

unique values in Status:
Status
C     10117
D      4525
CL      358
Name: count, dtype: int64



In [12]:
# check for skewness
print("Skewness of each column in numerical:")
for column in numerical_features:
    print(column, ":", train[column].skew())

Skewness of each column in numerical:
id : 0.0
N_Days : 5.723226588086048
Age : 0.2501326957918793
Bilirubin : 4.228548344977254
Cholesterol : 4.499543223972586
Albumin : -0.30018987991930035
Copper : 3.115699180745672
Alk_Phos : 3.1538951588374986
SGOT : 65.28128415449646
Tryglicerides : 2.368980642200841
Platelets : 2.340967199225039
Prothrombin : 1.2436597583866906
Stage : -0.4751521529128604


# 3. Compile Preprocessing Pipeline

All of the preprocessing classes or functions defined earlier will be compiled in this step.

If you use sklearn to create preprocessing classes, you can list your preprocessing classes in the Pipeline object sequentially, and then fit and transform your data.

In [ ]:
# from sklearn.pipeline import Pipeline

# # Note: You can add or delete preprocessing components from this pipeline

# pipe = Pipeline([("imputer", FeatureImputer()),
#                  ("featurecreator", FeatureCreator()),
#                  ("scaler", FeatureScaler()),
#                  ("encoder", FeatureEncoder())])

# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

In [ ]:
# # Your code should work up until this point
# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

or create your own here

In [ ]:
# Write your code here

# 4. Modeling and Validation

Modelling is the process of building your own machine learning models to solve specific problems, or in this assignment context, predicting the probability for each class in the `Status` feature (`Status_C`, `Status_CL`, `Status_D`). Validation is the process of evaluating your trained model using the validation set or cross-validation method and providing some metrics that can help you decide what to do in the next iteration of development.

## KNN

In [ ]:
# Type your code here

## Naive Bayes

In [ ]:
# Type your code here

## ID3

In [ ]:
# Type your code here

## SVM

In [ ]:
# Type your code here

## Logistic Regression

In [ ]:
# Type your code here

## Notes for improvements

- **Visualize the model evaluation result**

This will help you to understand the details more clearly about your model's performance. From the visualization, you can see clearly if your model is leaning towards a class than the others. (Hint: confusion matrix, ROC-AUC curve, etc.)

- **Explore the hyperparameters of your models**

Each models have their own hyperparameters. And each of the hyperparameter have different effects on the model behaviour. You can optimize the model performance by finding the good set of hyperparameters through a process called **hyperparameter tuning**. (Hint: Grid search, random search, bayesian optimization)

- **Cross-validation**

Cross-validation is a critical technique in machine learning and data science for evaluating and validating the performance of predictive models. It provides a more **robust** and **reliable** evaluation method compared to a hold-out (single train-test set) validation. Though, it requires more time and computing power because of how cross-validation works. (Hint: k-fold cross-validation, stratified k-fold cross-validation, etc.)

- **Ensemble methods**

Ensemble methods are powerful machine learning techniques that combine the predictions of multiple models (often referred to as base learners or weak learners) to create a stronger, more accurate predictive model. The idea behind ensemble methods is that by aggregating the opinions of multiple models, you can reduce the impact of individual model errors and improve overall prediction performance. (Hint: bagging, boosting, stacking, voting)

- **Model interpretation**

Model interpretation is the process of understanding and explaining the inner workings of a machine learning model, particularly its decision-making process. Interpretation helps data scientists, stakeholders, and end-users gain insights into why a model makes certain predictions or classifications. Model interpretation is crucial for building trust in machine learning systems, identifying biases, and extracting actionable information from models. (Hint: Feature importance, PDP, SHAP Values, etc)

- **Explore other models**

There are a lot of ML models that you can use in this usecase. Try to explore and use them to solve this problem.

## Submission
To predict the test set target feature and submit the results to the kaggle competition platform, do the following:
1. Create a new pipeline instance identical to the first in Data Preprocessing
2. With the pipeline, apply `fit_transform` to the original training set before splitting, then only apply `transform` to the test set.
3. Retrain the model on the preprocessed training set
4. Predict the test set
5. Make sure the submission contains the `id`, `Status_C`, `Status_CL`, `Status_D` column.

In [ ]:
# Type your code here

# 6. Error Analysis

Based on all the process you have done until the modeling and evaluation step, write an analysis to support each steps you have taken to solve this problem. Write the analysis using the markdown block. Some questions that may help you in writing the analysis:

- Does my model perform better in predicting one class than the other? If so, why is that?
- To each models I have tried, which performs the best and what could be the reason?
- Is it better for me to impute or drop the missing data? Why?
- Does feature scaling help improve my model performance?
- etc...

`Provide your analysis here`